# Classification Experiment: Followers
---
This Notebook, includes a series of experiments, on using a node's Followers for classification.

Libraries:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
import json
import tweepy
import time

from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import FeatureUnion

from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

import joblib
import time 

import seaborn as sns

Twitter API Authentication:

In [2]:
twitter_credentials = []
with open('../../../../twitter_credentials.json', 'r') as f:
    twitter_credentials = json.load(f)

auth = tweepy.OAuthHandler(twitter_credentials['consumer_key'], twitter_credentials['consumer_secret'])
auth.set_access_token(twitter_credentials['access_token_key'],twitter_credentials['access_token_secret'])
API = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, timeout=60*5)

In [3]:
# Function For Text Normalization
def clean_text(data):
    urls = r'http\S+'
    non_unicode_char = r'\W'
    numbers = r'[0-9_]'
    fix_whitespace = r'\s+'
    single_whitespace = ' '
    
    data = (data.replace([urls], single_whitespace, regex=True)
                    .replace([non_unicode_char, numbers], single_whitespace, regex=True)
                    .replace(fix_whitespace, single_whitespace, regex=True))
    data = data.apply(lambda s: s.lower() if type(s) == str else s)
    return data

nlp_el = spacy.load('el_core_news_md')
nlp_en = spacy.load('en_core_web_sm')
STOPWORDS = set(list(spacy.lang.en.STOP_WORDS) + list(spacy.lang.el.STOP_WORDS))

def remove_stopwords(row):
    row = [str(token) for token in nlp_el(row)]
    return [w for w in row if w not in STOPWORDS]

def tokenize_lemmatize(row):
    return [str(token.lemma_) for token in nlp_el(row)]

# Dataset
---

In [4]:
# Read Training Set
training_set = pd.read_csv('../../../../datasets/Greek Politicians/classification/parliament_members_training_set.csv')
training_set = training_set.replace(np.nan, '')
training_set.head()

,screen_name,name,description,statuses_count,recent_100_tweets,friends_count,recent_100_friends_nd,followers_count,default_profile_image,favourites_count,parliament_member,friends_politician_count_1000,followers_politician_count_1000,mentions_politician_count
0,evefthym,Ευάγγελος Ευθυμίου,Καναλάκι \nΠάργα \nΙωάννινα,135,#NewProfilePic https://t.co/h0wlMIS6U0 @wavyp...,156,Panagiotis Vougious Κύριε Μάτσιο ισχυρίζεται ...,448,False,696,0,5,3.0,0
1,Qdv6zNFe0nkBH2m,Χριστίνα Κεραμεως,Χιούμορ,0,,18,Kostas.Vaxevanis χρήστης του https://t.co/ArS...,6,False,3,0,3,0.0,0
2,michail_pana,Μιχαήλ Παναγιωτόπουλος,Απόστρατος Ναύαρχος του Πολέμικού Ναυτικού.\nΜ...,368,Ορθή και τεκμηριωμένη προσέγγιση. \nΣυγχαρητή...,70,Symban 300 UFC🇬🇷🇦🇲🇨🇾🇷🇸🇸🇦🇫🇷🇮🇱🇦🇹🇧🇬 Xara Stefano...,358,False,658,0,0,2.0,0
3,spirtzisforever,Χρήστος not Σπίρτζης,Parody account/Fan account του πιο ερωτικού βο...,7,"@NasosNot Επι ΠΑΣΟΚ είχαμε κλαδικές, αυτό ήτα...",7,Νάσος Ηλιόπουλος Εκπρόσωπος Τύπου του ΣΥΡΙΖΑ-...,4,False,2,0,1,0.0,2
4,FdZwMqKWXciTydT,Ειρήνη Γκαρα,,1,Καλό βράδυ!!!!,3,Kostas.Vaxevanis χρήστης του https://t.co/ArS...,0,False,0,0,0,0.0,0


# Only Counts

In [33]:
X = training_set
y = training_set.parliament_member

In [34]:
# Function to select the data
def get_data_(df):
    data = df['followers_politician_count_1000'].to_numpy()
    return data.reshape(-1,1)


get_data = FunctionTransformer(get_data_)


print('------------------ Support Vector Machine -------------------\n')

# The pipeline
pipeline = Pipeline([
    ('selector', get_data),
    ('svm', svm.SVC())
])

# Paramters for optimization
parameters = {'svm__C' : [0.1,0.5,1,5,10],
              'svm__kernel':['linear', 'poly', 'rbf', 'sigmoid']}

grid = GridSearchCV(pipeline, parameters, n_jobs = 4)
grid.fit(X, y)
    
print(f' Best Params: {grid.best_params_}.\n Score: {grid.best_score_}')


print('\n\n------------------ kNN -------------------\n')

# The pipeline
pipeline = Pipeline([
    ('selector', get_data),
    ('knn', KNeighborsClassifier())
])

# Paramters for optimization
parameters = {'knn__n_neighbors': [i for i in range(20)],
              'knn__weights': ['uniform', 'distance']}

grid = GridSearchCV(pipeline, parameters, n_jobs = 4)
grid.fit(X, y)
    
print(f' Best Params: {grid.best_params_}.\n Score: {grid.best_score_}')

print('\n\n------------------ Logistic Regression -------------------\n')

# The pipeline
pipeline = Pipeline([
    ('selector', get_data),
    ('lr', LogisticRegression(max_iter=1000))
])

# Paramters for optimization
parameters = {'lr__penalty': ['l1', 'l2', 'elasticnet'],
              'lr__C': [0.1, 0.5, 1, 5, 10]}

grid = GridSearchCV(pipeline, parameters, n_jobs = 4)
grid.fit(X, y)
    
print(f' Best Params: {grid.best_params_}.\n Score: {grid.best_score_}')

------------------ Support Vector Machine -------------------

 Best Params: {'svm__C': 0.1, 'svm__kernel': 'linear'}.
 Score: 0.7566666666666666


------------------ kNN -------------------



/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.52666667 0.52666667 0.68666667 0.69333333
 0.75333333 0.74       0.73666667 0.73       0.75666667 0.74333333
 0.74333333 0.73333333 0.76333333 0.75       0.75666667 0.75
 0.76333333 0.75       0.76333333 0.75       0.76333333 0.75
 0.77333333 0.76       0.77333333 0.76       0.77333333 0.76
 0.76       0.74666667 0.75666667 0.76       0.76333333 0.76
 0.75666667 0.76       0.76333333 0.76      ]
  warnings.warn(


 Best Params: {'knn__n_neighbors': 12, 'knn__weights': 'uniform'}.
 Score: 0.7733333333333332


------------------ Logistic Regression -------------------

 Best Params: {'lr__C': 0.1, 'lr__penalty': 'l2'}.
 Score: 0.7566666666666666


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan 0.75666667        nan        nan 0.75666667        nan
        nan 0.75666667        nan        nan 0.75666667        nan
        nan 0.75666667        nan]
  warnings.warn(


### Export Model

In [8]:
X = training_set
y = training_set.parliament_member

# Function to select the data
def get_data_parl_fo(df):
    data = df['followers_politician_count_1000'].to_numpy()
    return data.reshape(-1,1)


get_data = FunctionTransformer(get_data_parl_fo)

# The pipeline
pipeline = Pipeline([
    ('selector', get_data),
    ('knn', KNeighborsClassifier(n_neighbors=12, weights='uniform'))
])

pipeline.fit(X,y)

Pipeline(steps=[('selector',
                 FunctionTransformer(func=<function get_data_parl_fo at 0x7fa54f1d7040>)),
                ('knn', KNeighborsClassifier(n_neighbors=12))])

In [9]:
filename = 'classifier_parl_fo.sav'
joblib.dump(pipeline, filename)

['classifier_parl_fo.sav']

## Friends and Followers Politician COunt

In [35]:
X = training_set
y = training_set.parliament_member

In [37]:
# Function to select the data
def get_data_(df):
    data = df[['friends_politician_count_1000', 'followers_politician_count_1000']].to_numpy()
    return data


get_data = FunctionTransformer(get_data_)


print('------------------ Support Vector Machine -------------------\n')

# The pipeline
pipeline = Pipeline([
    ('selector', get_data),
    ('svm', svm.SVC())
])

# Paramters for optimization
parameters = {'svm__C' : [0.1,0.5,1,5,10],
              'svm__kernel':['linear', 'poly', 'rbf', 'sigmoid']}

grid = GridSearchCV(pipeline, parameters, n_jobs = 4)
grid.fit(X, y)
    
print(f' Best Params: {grid.best_params_}.\n Score: {grid.best_score_}')


print('\n\n------------------ kNN -------------------\n')

# The pipeline
pipeline = Pipeline([
    ('selector', get_data),
    ('knn', KNeighborsClassifier())
])

# Paramters for optimization
parameters = {'knn__n_neighbors': [i for i in range(20)],
              'knn__weights': ['uniform', 'distance']}

grid = GridSearchCV(pipeline, parameters, n_jobs = 4)
grid.fit(X, y)
    
print(f' Best Params: {grid.best_params_}.\n Score: {grid.best_score_}')

print('\n\n------------------ Logistic Regression -------------------\n')

# The pipeline
pipeline = Pipeline([
    ('selector', get_data),
    ('lr', LogisticRegression(max_iter=1000))
])

# Paramters for optimization
parameters = {'lr__penalty': ['l1', 'l2', 'elasticnet'],
              'lr__C': [0.1, 0.5, 1, 5, 10]}

grid = GridSearchCV(pipeline, parameters, n_jobs = 4)
grid.fit(X, y)
    
print(f' Best Params: {grid.best_params_}.\n Score: {grid.best_score_}')

------------------ Support Vector Machine -------------------

 Best Params: {'svm__C': 10, 'svm__kernel': 'rbf'}.
 Score: 0.9


------------------ kNN -------------------



/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.80666667 0.80666667 0.77       0.82666667
 0.89       0.86333333 0.87       0.86666667 0.89666667 0.88333333
 0.88666667 0.88333333 0.89       0.88333333 0.88666667 0.88333333
 0.89666667 0.88666667 0.89       0.88666667 0.89       0.89
 0.89       0.89       0.89       0.88       0.87666667 0.88333333
 0.88       0.87666667 0.87333333 0.88333333 0.87333333 0.88
 0.87333333 0.88       0.87333333 0.88      ]
  warnings.warn(


 Best Params: {'knn__n_neighbors': 5, 'knn__weights': 'uniform'}.
 Score: 0.8966666666666667


------------------ Logistic Regression -------------------

 Best Params: {'lr__C': 0.1, 'lr__penalty': 'l2'}.
 Score: 0.8666666666666668


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan 0.86666667        nan        nan 0.86666667        nan
        nan 0.86666667        nan        nan 0.86666667        nan
        nan 0.86666667        nan]
  warnings.warn(


### Export Model

In [47]:
X = training_set
y = training_set.parliament_member

# Function to select the data
def get_data_(df):
    data = df[['friends_politician_count_1000', 'followers_politician_count_1000']].to_numpy()
    return data


get_data = FunctionTransformer(get_data_)

# The pipeline
pipeline = Pipeline([
    ('selector', get_data),
    ('svm', svm.SVC(C=10, kernel='rbf'))
])

pipeline.fit(X,y)

Pipeline(steps=[('selector',
                 FunctionTransformer(func=<function get_data_ at 0x7f8c4d7d9160>)),
                ('svm', SVC(C=10))])

In [48]:
filename = '../classifiers/classifier_politician_frfo.sav'
joblib.dump(pipeline, filename)

['../classifiers/classifier_politician_frfo.sav']

# Name Description Tweets and Counts

## Without NLP

In [38]:
train = training_set.copy()
train['textdata'] = clean_text(train['name'] + ' ' + train['description'] + ' ' + train['recent_100_tweets'])
X = train
y = train.parliament_member

In [39]:
def get_text_data_(df):
    
    return df.textdata

get_text_data = FunctionTransformer(get_text_data_)

def get_numeric_data_(df):
    data = df[['friends_politician_count_1000', 'followers_politician_count_1000']].to_numpy()
    return data

get_numeric_data = FunctionTransformer(get_numeric_data_)




print('------------------ Support Vector Machine -------------------\n')

pipeline = Pipeline([
    ('features', FeatureUnion([
            ('numeric_features', Pipeline([
                ('selector_num', get_numeric_data)
            ])),
             ('text_features', Pipeline([
                ('selector_text', get_text_data),
                ('vectorizer', TfidfVectorizer()),
            ]))
         ])),
     ('svm', svm.SVC())
])


# Paramters for optimization
parameters = {'features__text_features__vectorizer__max_df': [0.5, 0.75, 1],
              'features__text_features__vectorizer__min_df': [1, 5, 10],
              'features__text_features__vectorizer__max_features': [1000, 2000, None],
              'svm__C' : [0.1,0.5,1,5,10],
              'svm__kernel':['linear', 'poly', 'rbf', 'sigmoid']
                  }

grid = GridSearchCV(pipeline, parameters, n_jobs = 4, verbose=1)
grid.fit(X, y)
    
print(f' Best Params: {grid.best_params_}.\n Score: {grid.best_score_}')


print('\n\n------------------ kNN -------------------\n')

pipeline = Pipeline([
    ('features', FeatureUnion([
            ('numeric_features', Pipeline([
                ('selector_num', get_numeric_data)
            ])),
             ('text_features', Pipeline([
                ('selector_text', get_text_data),
                ('vectorizer', TfidfVectorizer()),
            ]))
         ])),
     ('knn', KNeighborsClassifier())
])


# Paramters for optimization
parameters = {'features__text_features__vectorizer__max_df': [0.5, 0.75, 1],
              'features__text_features__vectorizer__min_df': [1, 5, 10],
              'features__text_features__vectorizer__max_features': [1000, 2000, None],
              'knn__n_neighbors': [1,2,3,4,5,6,7,8,9,10],
              'knn__weights': ['uniform', 'distance']
                  }

grid = GridSearchCV(pipeline, parameters, n_jobs = 4, verbose=1)
grid.fit(X, y)
    
print(f' Best Params: {grid.best_params_}.\n Score: {grid.best_score_}')


print('\n\n------------------ Logistic Regression -------------------\n')


pipeline = Pipeline([
    ('features', FeatureUnion([
            ('numeric_features', Pipeline([
                ('selector_num', get_numeric_data)
            ])),
             ('text_features', Pipeline([
                ('selector_text', get_text_data),
                ('vectorizer', TfidfVectorizer()),
            ]))
         ])),
     ('lr', LogisticRegression(max_iter=1000))
])


# Paramters for optimization
parameters = {'features__text_features__vectorizer__max_df': [0.5, 0.75, 1],
              'features__text_features__vectorizer__min_df': [1, 5, 10],
              'features__text_features__vectorizer__max_features': [1000, 2000, None],
              'lr__penalty': ['l1', 'l2', 'elasticnet', 'none'],
              'lr__C': [0.1, 0.5, 1, 5, 10]
                  }

grid = GridSearchCV(pipeline, parameters, n_jobs = 4, verbose=1)
grid.fit(X, y)
    
print(f' Best Params: {grid.best_params_}.\n Score: {grid.best_score_}')

del train

------------------ Support Vector Machine -------------------

Fitting 5 folds for each of 540 candidates, totalling 2700 fits


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.88       0.75       0.85       0.85333333 0.91       0.75666667
 0.86666667 0.83333333 0.90333333 0.76333333 0.87666667 0.83333333
 0.90333333 0.77666667 0.88       0.83333333 0.90333333 0.77666667
 0.89333333 0.83       0.88       0.75       0.85       0.85333333
 0.90666667 0.75666667 0.86666667 0.83333333 0.90666667 0.76333333
 0.87666667 0.83333333 0.90333333 0.77666667 0.88       0.83333333
 0.89333333 0.77666667 0.89333333 0.83       0.88       0.75
 0.85       0.85333333 0.90666667 0.75666667 0.86666667 0.83333333
 0.91       0.76333333 0.87666667 0.83333333 0.88666667 0.77666667
 0.88       0.83333333 0.88666667 0.77666667 0.89333333 0.83
 0.88       0.75       0.85       0.85333333 0.91       0.75666667
 0.86666667 0.83333333 0.90666667 0.76333333 0.87666667 0.83333333
 0.89333333 0.77666667 0.88   

 Best Params: {'features__text_features__vectorizer__max_df': 0.75, 'features__text_features__vectorizer__max_features': 1000, 'features__text_features__vectorizer__min_df': 10, 'svm__C': 1, 'svm__kernel': 'linear'}.
 Score: 0.9133333333333333


------------------ kNN -------------------

Fitting 5 folds for each of 540 candidates, totalling 2700 fits


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.83       0.83       0.78       0.83       0.89       0.88
 0.87333333 0.88       0.9        0.89333333 0.89       0.9
 0.88666667 0.89333333 0.88666667 0.89       0.89666667 0.9
 0.89       0.89333333 0.83       0.83       0.78       0.83
 0.89333333 0.88333333 0.87333333 0.88       0.89666667 0.89333333
 0.89       0.9        0.88666667 0.89333333 0.88666667 0.89333333
 0.89666667 0.9        0.89       0.89333333 0.83       0.83
 0.78       0.83       0.89333333 0.88333333 0.87333333 0.88
 0.89333333 0.89333333 0.89       0.9        0.88666667 0.89333333
 0.88666667 0.89       0.89666667 0.89666667 0.89       0.89333333
 0.82666667 0.82666667 0.78       0.82666667 0.89333333 0.88333333
 0.87333333 0.88       0.89666667 0.89666667 0.89       0.9
 0.88666667 0.89333333 0.88666667 0.89333333 0.89666667 0.9
 0.

 Best Params: {'features__text_features__vectorizer__max_df': 1, 'features__text_features__vectorizer__max_features': None, 'features__text_features__vectorizer__min_df': 1, 'knn__n_neighbors': 9, 'knn__weights': 'distance'}.
 Score: 0.9033333333333333


------------------ Logistic Regression -------------------

Fitting 5 folds for each of 540 candidates, totalling 2700 fits


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan 0.87              nan 0.89666667        nan 0.88
        nan 0.89666667        nan 0.9               nan 0.89666667
        nan 0.91              nan 0.89666667        nan 0.91333333
        nan 0.89666667        nan 0.87              nan 0.89666667
        nan 0.88              nan 0.89666667        nan 0.9
        nan 0.89666667        nan 0.91333333        nan 0.89666667
        nan 0.91              nan 0.89666667        nan 0.87
        nan 0.9               nan 0.88333333        nan 0.9
        nan 0.89666667        nan 0.9               nan 0.91333333
        nan 0.9               nan 0.91333333        nan 0.9
        nan 0.87              nan 0.91              nan 0.88333333
        nan 0.91              nan 0.9               nan 0.91
        nan 0.91333333        nan 0.91              nan 0

 Best Params: {'features__text_features__vectorizer__max_df': 0.5, 'features__text_features__vectorizer__max_features': 1000, 'features__text_features__vectorizer__min_df': 1, 'lr__C': 10, 'lr__penalty': 'l2'}.
 Score: 0.9133333333333334


## Stop Word Removal

In [40]:
train = training_set.copy()
train['textdata'] = clean_text(train['name'] + ' ' + train['description'] + ' ' + train['recent_100_tweets'])
train['textdata'] = train['textdata'].apply(lambda row: remove_stopwords(row))
train['textdata'] = train['textdata'].apply(lambda row: ' '.join(row))

X = train
y = train.parliament_member

In [41]:
def get_text_data_(df):
    
    return df.textdata

get_text_data = FunctionTransformer(get_text_data_)


def get_numeric_data_(df):
    data = df[['friends_politician_count_1000', 'followers_politician_count_1000']].to_numpy()
    return data

get_numeric_data = FunctionTransformer(get_numeric_data_)




print('------------------ Support Vector Machine -------------------\n')

pipeline = Pipeline([
    ('features', FeatureUnion([
            ('numeric_features', Pipeline([
                ('selector_num', get_numeric_data)
            ])),
             ('text_features', Pipeline([
                ('selector_text', get_text_data),
                ('vectorizer', TfidfVectorizer()),
            ]))
         ])),
     ('svm', svm.SVC())
])


# Paramters for optimization
parameters = {'features__text_features__vectorizer__max_df': [0.5, 0.75, 1],
              'features__text_features__vectorizer__min_df': [1, 5, 10],
              'features__text_features__vectorizer__max_features': [1000, 2000, None],
              'svm__C' : [0.1,0.5,1,5,10],
              'svm__kernel':['linear', 'poly', 'rbf', 'sigmoid']
                  }

grid = GridSearchCV(pipeline, parameters, n_jobs = 4, verbose=1)
grid.fit(X, y)
    
print(f' Best Params: {grid.best_params_}.\n Score: {grid.best_score_}')


print('\n\n------------------ kNN -------------------\n')

pipeline = Pipeline([
    ('features', FeatureUnion([
            ('numeric_features', Pipeline([
                ('selector_num', get_numeric_data)
            ])),
             ('text_features', Pipeline([
                ('selector_text', get_text_data),
                ('vectorizer', TfidfVectorizer()),
            ]))
         ])),
     ('knn', KNeighborsClassifier())
])


# Paramters for optimization
parameters = {'features__text_features__vectorizer__max_df': [0.5, 0.75, 1],
              'features__text_features__vectorizer__min_df': [1, 5, 10],
              'features__text_features__vectorizer__max_features': [1000, 2000, None],
              'knn__n_neighbors': [1,2,3,4,5,6,7,8,9,10],
              'knn__weights': ['uniform', 'distance']
                  }

grid = GridSearchCV(pipeline, parameters, n_jobs = 4, verbose=1)
grid.fit(X, y)
    
print(f' Best Params: {grid.best_params_}.\n Score: {grid.best_score_}')


print('\n\n------------------ Logistic Regression -------------------\n')


pipeline = Pipeline([
    ('features', FeatureUnion([
            ('numeric_features', Pipeline([
                ('selector_num', get_numeric_data)
            ])),
             ('text_features', Pipeline([
                ('selector_text', get_text_data),
                ('vectorizer', TfidfVectorizer()),
            ]))
         ])),
     ('lr', LogisticRegression(max_iter=1000))
])


# Paramters for optimization
parameters = {'features__text_features__vectorizer__max_df': [0.5, 0.75, 1],
              'features__text_features__vectorizer__min_df': [1, 5, 10],
              'features__text_features__vectorizer__max_features': [1000, 2000, None],
              'lr__penalty': ['l1', 'l2', 'elasticnet', 'none'],
              'lr__C': [0.1, 0.5, 1, 5, 10]
                  }

grid = GridSearchCV(pipeline, parameters, n_jobs = 4, verbose=1)
grid.fit(X, y)
    
print(f' Best Params: {grid.best_params_}.\n Score: {grid.best_score_}')

del train

------------------ Support Vector Machine -------------------

Fitting 5 folds for each of 540 candidates, totalling 2700 fits


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.88       0.75       0.85       0.85333333 0.91       0.75666667
 0.86666667 0.83333333 0.90333333 0.76333333 0.87666667 0.83333333
 0.91333333 0.77666667 0.88       0.83333333 0.91666667 0.77666667
 0.89333333 0.83       0.88       0.75       0.85       0.85333333
 0.91       0.75666667 0.86666667 0.83333333 0.90333333 0.76333333
 0.87666667 0.83333333 0.90333333 0.77666667 0.88       0.83333333
 0.9        0.77666667 0.89333333 0.83       0.88       0.75
 0.85       0.85333333 0.91       0.75666667 0.86666667 0.83333333
 0.91       0.76333333 0.87666667 0.83333333 0.90666667 0.77666667
 0.88       0.83333333 0.91666667 0.77666667 0.89333333 0.83
 0.88       0.75       0.85       0.85333333 0.90666667 0.75666667
 0.86666667 0.83333333 0.90333333 0.76333333 0.87666667 0.83333333
 0.90333333 0.77666667 0.88   

 Best Params: {'features__text_features__vectorizer__max_df': 0.75, 'features__text_features__vectorizer__max_features': 1000, 'features__text_features__vectorizer__min_df': 1, 'svm__C': 5, 'svm__kernel': 'linear'}.
 Score: 0.9199999999999999


------------------ kNN -------------------

Fitting 5 folds for each of 540 candidates, totalling 2700 fits


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.83333333 0.83333333 0.78       0.83333333 0.89       0.88
 0.87333333 0.88       0.89666667 0.89666667 0.89       0.9
 0.89       0.89666667 0.88666667 0.89       0.89666667 0.9
 0.89       0.89333333 0.83       0.83       0.78       0.83
 0.89333333 0.88333333 0.87333333 0.88       0.9        0.89666667
 0.89       0.9        0.89       0.89666667 0.88666667 0.89
 0.89666667 0.9        0.89       0.89333333 0.83       0.83
 0.78       0.83       0.89333333 0.88333333 0.87333333 0.88
 0.9        0.89666667 0.89       0.9        0.89       0.89666667
 0.88666667 0.89333333 0.89666667 0.9        0.89       0.89333333
 0.83       0.83       0.78       0.83       0.89       0.88
 0.87666667 0.88       0.89333333 0.89333333 0.89       0.9
 0.89       0.89666667 0.88666667 0.89333333 0.89666667 0.9
 0.89       0.8

 Best Params: {'features__text_features__vectorizer__max_df': 1, 'features__text_features__vectorizer__max_features': None, 'features__text_features__vectorizer__min_df': 1, 'knn__n_neighbors': 9, 'knn__weights': 'distance'}.
 Score: 0.9033333333333333


------------------ Logistic Regression -------------------

Fitting 5 folds for each of 540 candidates, totalling 2700 fits


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan 0.87              nan 0.90666667        nan 0.88
        nan 0.90666667        nan 0.89666667        nan 0.90666667
        nan 0.91              nan 0.90666667        nan 0.91
        nan 0.90666667        nan 0.87              nan 0.91666667
        nan 0.88              nan 0.91666667        nan 0.9
        nan 0.91666667        nan 0.91333333        nan 0.91666667
        nan 0.91333333        nan 0.91666667        nan 0.87
        nan 0.9               nan 0.88              nan 0.9
        nan 0.89666667        nan 0.9               nan 0.91666667
        nan 0.9               nan 0.91333333        nan 0.9
        nan 0.87              nan 0.89666667        nan 0.88333333
        nan 0.89666667        nan 0.9               nan 0.89666667
        nan 0.91              nan 0.89666667        nan 0

 Best Params: {'features__text_features__vectorizer__max_df': 0.5, 'features__text_features__vectorizer__max_features': 2000, 'features__text_features__vectorizer__min_df': 10, 'lr__C': 10, 'lr__penalty': 'l2'}.
 Score: 0.9200000000000002


## Lemmatization and Stop Word Removal

In [42]:
train = training_set.copy()
train['textdata'] = clean_text(train['name'] + ' ' + train['description'] + ' ' + train['recent_100_tweets'])
train['textdata'] = train['textdata'].apply(lambda row: tokenize_lemmatize(row))
train['textdata'] = train['textdata'].apply(lambda row: ' '.join(row))
train['textdata'] = train['textdata'].apply(lambda row: remove_stopwords(row))
train['textdata'] = train['textdata'].apply(lambda row: ' '.join(row))
X = train
y = train.parliament_member

In [43]:
def get_text_data_(df):
    return df.textdata

get_text_data = FunctionTransformer(get_text_data_)


def get_numeric_data_(df):
    data = df[['friends_politician_count_1000', 'followers_politician_count_1000']].to_numpy()
    return data

get_numeric_data = FunctionTransformer(get_numeric_data_)




print('------------------ Support Vector Machine -------------------\n')

pipeline = Pipeline([
    ('features', FeatureUnion([
            ('numeric_features', Pipeline([
                ('selector_num', get_numeric_data)
            ])),
             ('text_features', Pipeline([
                ('selector_text', get_text_data),
                ('vectorizer', TfidfVectorizer()),
            ]))
         ])),
     ('svm', svm.SVC())
])


# Paramters for optimization
parameters = {'features__text_features__vectorizer__max_df': [0.5, 0.75, 1],
              'features__text_features__vectorizer__min_df': [1, 5, 10],
              'features__text_features__vectorizer__max_features': [1000, 2000, None],
              'svm__C' : [0.1,0.5,1,5,10],
              'svm__kernel':['linear', 'poly', 'rbf', 'sigmoid']
                  }

grid = GridSearchCV(pipeline, parameters, n_jobs = 4, verbose=1)
grid.fit(X, y)
    
print(f' Best Params: {grid.best_params_}.\n Score: {grid.best_score_}')


print('\n\n------------------ kNN -------------------\n')

pipeline = Pipeline([
    ('features', FeatureUnion([
            ('numeric_features', Pipeline([
                ('selector_num', get_numeric_data)
            ])),
             ('text_features', Pipeline([
                ('selector_text', get_text_data),
                ('vectorizer', TfidfVectorizer()),
            ]))
         ])),
     ('knn', KNeighborsClassifier())
])


# Paramters for optimization
parameters = {'features__text_features__vectorizer__max_df': [0.5, 0.75, 1],
              'features__text_features__vectorizer__min_df': [1, 5, 10],
              'features__text_features__vectorizer__max_features': [1000, 2000, None],
              'knn__n_neighbors': [1,2,3,4,5,6,7,8,9,10],
              'knn__weights': ['uniform', 'distance']
                  }

grid = GridSearchCV(pipeline, parameters, n_jobs = 4, verbose=1)
grid.fit(X, y)
    
print(f' Best Params: {grid.best_params_}.\n Score: {grid.best_score_}')


print('\n\n------------------ Logistic Regression -------------------\n')


pipeline = Pipeline([
    ('features', FeatureUnion([
            ('numeric_features', Pipeline([
                ('selector_num', get_numeric_data)
            ])),
             ('text_features', Pipeline([
                ('selector_text', get_text_data),
                ('vectorizer', TfidfVectorizer()),
            ]))
         ])),
     ('lr', LogisticRegression(max_iter=1000))
])


# Paramters for optimization
parameters = {'features__text_features__vectorizer__max_df': [0.5, 0.75, 1],
              'features__text_features__vectorizer__min_df': [1, 5, 10],
              'features__text_features__vectorizer__max_features': [1000, 2000, None],
              'lr__penalty': ['l1', 'l2', 'elasticnet', 'none'],
              'lr__C': [0.1, 0.5, 1, 5, 10]
                  }

grid = GridSearchCV(pipeline, parameters, n_jobs = 4, verbose=1)
grid.fit(X, y)
    
print(f' Best Params: {grid.best_params_}.\n Score: {grid.best_score_}')

del train

------------------ Support Vector Machine -------------------

Fitting 5 folds for each of 540 candidates, totalling 2700 fits


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.88       0.75       0.85       0.85333333 0.91333333 0.75666667
 0.86666667 0.83333333 0.91       0.76333333 0.87666667 0.83333333
 0.90333333 0.77666667 0.88       0.83333333 0.91       0.77666667
 0.89333333 0.83       0.88       0.75       0.85       0.85333333
 0.91       0.75666667 0.86666667 0.83333333 0.90666667 0.76333333
 0.87666667 0.83333333 0.90333333 0.77666667 0.88       0.83333333
 0.90666667 0.77666667 0.89333333 0.83       0.88       0.75
 0.85       0.85333333 0.91       0.75666667 0.86666667 0.83333333
 0.91333333 0.76333333 0.87666667 0.83333333 0.89666667 0.77666667
 0.88       0.83333333 0.89333333 0.77666667 0.89333333 0.83
 0.88       0.75       0.85       0.85333333 0.90666667 0.75666667
 0.86666667 0.83333333 0.90333333 0.76333333 0.87666667 0.83333333
 0.90333333 0.77666667 0.88   

 Best Params: {'features__text_features__vectorizer__max_df': 0.5, 'features__text_features__vectorizer__max_features': 1000, 'features__text_features__vectorizer__min_df': 1, 'svm__C': 0.5, 'svm__kernel': 'linear'}.
 Score: 0.9133333333333333


------------------ kNN -------------------

Fitting 5 folds for each of 540 candidates, totalling 2700 fits


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.82333333 0.82333333 0.78       0.82333333 0.88666667 0.87666667
 0.87333333 0.87333333 0.89333333 0.89       0.89       0.89666667
 0.88666667 0.89333333 0.88666667 0.89333333 0.89666667 0.90333333
 0.89       0.90333333 0.81666667 0.81666667 0.78       0.81666667
 0.88666667 0.87666667 0.87333333 0.87333333 0.89333333 0.89
 0.89       0.9        0.88666667 0.89333333 0.88666667 0.89666667
 0.89666667 0.90333333 0.89       0.90333333 0.81666667 0.81666667
 0.78       0.81666667 0.88666667 0.87666667 0.87333333 0.87333333
 0.89333333 0.89       0.89       0.9        0.88666667 0.89333333
 0.88666667 0.89666667 0.89666667 0.90333333 0.89       0.9
 0.82       0.82       0.78       0.82       0.88666667 0.87666667
 0.87       0.87333333 0.89       0.89       0.89       0.89666667
 0.88666667 0.89333333 0.886666

 Best Params: {'features__text_features__vectorizer__max_df': 0.5, 'features__text_features__vectorizer__max_features': 1000, 'features__text_features__vectorizer__min_df': 1, 'knn__n_neighbors': 9, 'knn__weights': 'distance'}.
 Score: 0.9033333333333333


------------------ Logistic Regression -------------------

Fitting 5 folds for each of 540 candidates, totalling 2700 fits


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan 0.87333333        nan 0.89333333        nan 0.88333333
        nan 0.89333333        nan 0.90666667        nan 0.89333333
        nan 0.91              nan 0.89333333        nan 0.90333333
        nan 0.89333333        nan 0.87333333        nan 0.90333333
        nan 0.88333333        nan 0.90333333        nan 0.90333333
        nan 0.90333333        nan 0.91              nan 0.90333333
        nan 0.90666667        nan 0.90333333        nan 0.87333333
        nan 0.9               nan 0.88333333        nan 0.9
        nan 0.90333333        nan 0.9               nan 0.91
        nan 0.9               nan 0.90333333        nan 0.9
        nan 0.87              nan 0.88666667        nan 0.88333333
        nan 0.88666667        nan 0.9               nan 0.88666667
        nan 0.91              nan 0.88

 Best Params: {'features__text_features__vectorizer__max_df': 0.5, 'features__text_features__vectorizer__max_features': None, 'features__text_features__vectorizer__min_df': 5, 'lr__C': 5, 'lr__penalty': 'l2'}.
 Score: 0.9166666666666667


### Export Model

In [49]:
X = training_set
y = training_set.parliament_member

In [50]:
def get_text_data_(df):
    df = df.copy()
    df['textdata'] = clean_text(df['name']+ ' ' + df['description'] + ' ' + df['recent_100_tweets'])
    df['textdata'] = df['textdata'].apply(lambda row: remove_stopwords(row))
    df['textdata'] = df['textdata'].apply(lambda row: ' '.join(row))
    return df.textdata

get_text_data = FunctionTransformer(get_text_data_)


def get_numeric_data_(df):
    data = df[['friends_politician_count_1000', 'followers_politician_count_1000']].to_numpy()
    return data

get_numeric_data = FunctionTransformer(get_numeric_data_)


pipeline = Pipeline([
    ('features', FeatureUnion([
            ('numeric_features', Pipeline([
                ('selector_num', get_numeric_data)
            ])),
             ('text_features', Pipeline([
                ('selector_text', get_text_data),
                ('vectorizer', TfidfVectorizer(max_df=0.5, max_features=2000, min_df=10)),
            ]))
         ])),
     ('lr', LogisticRegression(max_iter=1000, penalty='l2',  C=10))
])


pipeline.fit(X, y)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('numeric_features',
                                                 Pipeline(steps=[('selector_num',
                                                                  FunctionTransformer(func=<function get_numeric_data_ at 0x7f8c4d7d9550>))])),
                                                ('text_features',
                                                 Pipeline(steps=[('selector_text',
                                                                  FunctionTransformer(func=<function get_text_data_ at 0x7f8c4d7d95e0>)),
                                                                 ('vectorizer',
                                                                  TfidfVectorizer(max_df=0.5,
                                                                                  max_features=2000,
                                                                                  min_df=10))]))])),
             

In [51]:
filename = '../classifiers/classifier_politician_ndtfrfo.sav'
joblib.dump(pipeline, filename)

['../classifiers/classifier_politician_ndtfrfo.sav']